In [ ]:
from gbfs.services import SystemDiscoveryService
import pandas as pd

In [ ]:
ds = SystemDiscoveryService()

In [ ]:
[x.get('System ID') for x in ds.systems if 'Los Angeles, CA' in x.get('Location')]

In [ ]:
[x.get('System ID') for x in ds.systems if 'San Diego, CA' in x.get('Location')]

In [ ]:
client = ds.instantiate_client('spin uc_san_diego')

In [ ]:
client.feed_names

In [ ]:
result = client.request_feed('free_bike_status')

In [ ]:
result["last_updated"]

In [ ]:
pd.DataFrame(result["data"]["bikes"])

In [ ]:
from glob import glob

In [ ]:
data = []
for i in glob("SanDiegoGBFS/*.csv"):
    x = pd.read_csv(i).set_index("bike_id")
    timestamp = pd.to_datetime(i.split("/")[1].split(".")[0])
    x[timestamp] = x.apply(lambda x: (x["lat"], x["lon"]), axis=1)
    data.append(x[[timestamp]])

aggregated_data = pd.concat(data, axis=1).T.sort_index()

In [ ]:
aggregated_data

In [ ]:
def get_distance(df):
    distances = []
    values = df.dropna()
    for a,b in zip(values[:-1], values[1:]):
        distances.append([a,b,distance.distance(a,b).m])
    new_df = pd.DataFrame(distances, columns=["Start","End","Distance"])
    return new_df[new_df.Distance > 30]

all_trips = []
for scooter in aggregated_data.columns:
    scooter_trace = aggregated_data[scooter].copy()
    trips = get_distance(scooter_trace)
    if len(trips) > 0:
        all_trips.append(trips)


In [ ]:
all_trips = pd.concat(all_trips)

In [ ]:
all_trips.Distance.hist(bins=100)

In [ ]:
import mplleaflet
from matplotlib import pyplot as plt

fig = plt.figure(figsize=[20,20])
ax = fig.gca()
for scooter in aggregated_data.columns[:10]:
    tmp = aggregated_data[[scooter]].copy().dropna().drop_duplicates()
    
    ax = tmp.plot.line(x="lon", y="lat",style='-o', ax=ax)
mplleaflet.display()

In [ ]:
trips = []
for scooter in aggregated_data.columns:
    tmp = aggregated_data[[scooter]].copy()
    if (tmp.isna().any()[0]):
        tmp = aggregated_data[[scooter]].copy().dropna().drop_duplicates()
        tmp["lat"] = tmp.apply(lambda x: x[0][0],axis=1)
        tmp["lon"] = tmp.apply(lambda x: x[0][1],axis=1)
        ddf = tmp.copy()
        ddf["lat_1"] = ddf["lat"].shift(-1)
        ddf["lon_1"] = ddf["lon"].shift(-1)
        ddf = ddf.dropna()
        if len(ddf) > 0:
            ddf["distance"] = ddf.apply(lambda x: distance.distance([x["lat"],x["lon"]],[x["lat_1"],x["lon_1"]]).m, axis=1)
            if((ddf["distance"]>10).any()):
                ax = ddf.plot.line(x="lon", y="lat",style='-o', ax=ax)
                tmp = ddf[ddf.distance>10].reset_index()[["index","lat","lon","lat_1","lon_1","distance"]]
                for i, r in tmp.iterrows():
                    tmp = r.to_dict()
                    tmp["bike"] =  ddf.columns[0]
                    trips.append(tmp)

In [ ]:
trips

In [ ]:
pd.DataFrame(trips)

In [ ]:
from gbfs.services import SystemDiscoveryService
import pandas as pd
from time import sleep

ds = SystemDiscoveryService()
providers = ['lyft_san', 'Link_San_Diego', 'spin san_diego', 'spin uc_san_diego']
clients = [[provider, ds.instantiate_client(provider)] for provider in providers]
while(True):
    for provider, client in clients:
        result = client.request_feed('free_bike_status')
        df = pd.DataFrame(result["data"]["bikes"])
        df.to_csv(f'{provider}_{result["last_updated"]}.csv.tar.gz')
    sleep(30)